In [2]:
# Neural Networks Training Script (a la Chisholm)

# NEURAL NETWORK TRAINING

# IMPORTING THE LIBRARIES

import numpy as np
import pandas as pd
from pandas import DataFrame as df
from sklearn.preprocessing import OneHotEncoder, MaxAbsScaler
from sklearn.compose import make_column_transformer
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import KFold, train_test_split
from random import shuffle

In [3]:
# DATA PRE-PROCESSING

# pre-process input variables

X = pd.read_csv('AxaltaData_210318_clean_070821.csv', usecols = [
        'wt_IPDA', 'wt_HDDA', 'wt_TMPTA', 'wt_TMPEOTA',
        'wt_DEM', 'wt_DBM', 'wt_DOM', 'wt_IPDI'])

In [4]:
# View at first three entries of each input variable X

print(X.head(3))

   wt_IPDA  wt_HDDA  wt_TMPTA  wt_TMPEOTA  wt_DEM  wt_DBM  wt_DOM  wt_IPDI
0     0.22     0.00      0.00        0.04     0.0     0.0    0.64      0.1
1     0.23     0.00      0.03        0.00     0.0     0.0    0.65      0.1
2     0.22     0.03      0.00        0.00     0.0     0.0    0.64      0.1


In [5]:
# Create X scaler array for scaling X inputs.

X_scaler = pd.read_csv('scaler_AxaltaData_210318_clean_070821.csv', usecols = [
        'wt_IPDA', 'wt_HDDA', 'wt_TMPTA', 'wt_TMPEOTA',
        'wt_DEM', 'wt_DBM', 'wt_DOM', 'wt_IPDI'])

In [6]:
# View at first three entries of each input variable X

print(X_scaler.head(3))

   wt_IPDA  wt_HDDA  wt_TMPTA  wt_TMPEOTA  wt_DEM  wt_DBM  wt_DOM  wt_IPDI
0        1        1         1           1       1       1       1        1
1        1        1         1           1       1       1       1        1
2        1        1         1           1       1       1       1        1


In [7]:
# Define the preprocessor for scaling and encoding the X data

X_preprocessor = make_column_transformer(
        (MaxAbsScaler(),
        ['wt_IPDA', 'wt_HDDA', 'wt_TMPTA', 'wt_TMPEOTA',
        'wt_DEM', 'wt_DBM', 'wt_DOM', 'wt_IPDI']))

In [8]:
# View the defined preprocessor

print(X_preprocessor)

ColumnTransformer(transformers=[('maxabsscaler', MaxAbsScaler(),
                                 ['wt_IPDA', 'wt_HDDA', 'wt_TMPTA',
                                  'wt_TMPEOTA', 'wt_DEM', 'wt_DBM', 'wt_DOM',
                                  'wt_IPDI'])])


In [9]:
# Add the scaler dataset to the current dataset.
# Doing this ensures consistent scaling of features.
# Also allows encoding of categorical variables if available.

X_w_scaler = X_preprocessor.fit_transform(X.append(X_scaler, sort = 'false'))

In [10]:
# Take a look at X w/ scaler data added:

print(X_w_scaler)

[[0.22 0.   0.   ... 0.   0.64 0.1 ]
 [0.23 0.   0.03 ... 0.   0.65 0.1 ]
 [0.22 0.03 0.   ... 0.   0.64 0.1 ]
 ...
 [1.   1.   1.   ... 1.   1.   1.  ]
 [1.   1.   1.   ... 1.   1.   1.  ]
 [1.   1.   1.   ... 1.   1.   1.  ]]


In [11]:
# Load the model outputs (Y).

# For now, use actual values in data and employ scaler before model fitting. 

# OPTION: USE NORM VALUES AND BYPASS SCALER

# Y ouputs: fisher_2h, fisher_24h, fisher_7d, DOI_1day, gloss_1day, DOI_7d, gloss_7d, Amtec_afterScratch, Amtec_afterReflow, 
# Amtect_afterScratchRel6600, Amtec_afterReflowRel6600


Y = pd.read_csv('AxaltaData_210318_clean_070821.csv', usecols = [
                'fisher_2h', 'fisher_24h', 'fisher_7d', 'DOI_1day',
                'gloss_1day', 'DOI_7d', 'gloss_7d',
                'Amtec_afterScratch', 'Amtec_afterReflow', 
                'Amtect_afterScratchRel6600', 'Amtec_afterReflowRel6600'])

In [12]:
# Create Y scaler array for scaling X inputs.

Y_scaler = pd.read_csv('scaler_AxaltaData_210318_clean_070821.csv', usecols = [
        'fisher_2h', 'fisher_24h', 'fisher_7d', 'DOI_1day',
        'gloss_1day', 'DOI_7d', 'gloss_7d',
        'Amtec_afterScratch', 'Amtec_afterReflow', 
        'Amtect_afterScratchRel6600', 'Amtec_afterReflowRel6600'])

In [13]:
# Define the preprocessor for scaling and encoding the Y data

Y_preprocessor = make_column_transformer(
        (MaxAbsScaler(),
         ['fisher_2h', 'fisher_24h', 'fisher_7d', 'DOI_1day',
        'gloss_1day', 'DOI_7d', 'gloss_7d',
        'Amtec_afterScratch', 'Amtec_afterReflow', 
        'Amtect_afterScratchRel6600', 'Amtec_afterReflowRel6600']))

In [14]:
# View the defined preprocessor

print(Y_preprocessor)

ColumnTransformer(transformers=[('maxabsscaler', MaxAbsScaler(),
                                 ['fisher_2h', 'fisher_24h', 'fisher_7d',
                                  'DOI_1day', 'gloss_1day', 'DOI_7d',
                                  'gloss_7d', 'Amtec_afterScratch',
                                  'Amtec_afterReflow',
                                  'Amtect_afterScratchRel6600',
                                  'Amtec_afterReflowRel6600'])])


In [15]:
# Add the scaler dataset to the current dataset.
# Doing this ensures consistent scaling of features.
# Also allows encoding of categorical variables if available.

Y_w_scaler = Y_preprocessor.fit_transform(Y.append(Y_scaler, sort = 'false'))

In [16]:
# Take a look at Y w/ scaler data added:

print(Y_w_scaler)

[[0.115   0.36475 0.4695  ... 0.6163  0.365   0.445  ]
 [0.0935  0.36375 0.458   ... 0.593   0.365   0.43   ]
 [0.08275 0.36575 0.46625 ... 0.6744  0.375   0.49   ]
 ...
 [1.      1.      1.      ... 1.      1.      1.     ]
 [1.      1.      1.      ... 1.      1.      1.     ]
 [1.      1.      1.      ... 1.      1.      1.     ]]


In [17]:
# Now remove the scaler data from the X and Y datasets.

X_checked =  X_w_scaler[0:(len(X_w_scaler)-len(X_scaler)),:]
Y_checked =  Y_w_scaler[0:(len(Y_w_scaler)-len(Y_scaler)),:]

In [18]:
# print(X_checked)

In [19]:
print(Y_checked)

[[0.115   0.36475 0.4695  0.933   0.842   0.908   0.852   0.4302  0.6163
  0.365   0.445  ]
 [0.0935  0.36375 0.458   0.929   0.853   0.915   0.849   0.4302  0.593
  0.365   0.43   ]
 [0.08275 0.36575 0.46625 0.938   0.854   0.912   0.854   0.4419  0.6744
  0.375   0.49   ]
 [0.02975 0.1435  0.24    0.941   0.819   0.895   0.817   0.4118  0.4706
  0.315   0.325  ]
 [0.02425 0.1294  0.21325 0.933   0.834   0.884   0.831   0.5059  0.5294
  0.39    0.37   ]
 [0.022   0.12075 0.20475 0.947   0.844   0.906   0.842   0.5765  0.6235
  0.445   0.435  ]
 [0.04875 0.17775 0.284   0.835   0.841   0.84    0.835   0.5647  0.6824
  0.435   0.475  ]
 [0.037   0.17875 0.26425 0.866   0.829   0.845   0.819   0.5294  0.6118
  0.405   0.425  ]
 [0.02425 0.14025 0.23925 0.831   0.845   0.843   0.843   0.6133  0.68
  0.47    0.47   ]
 [0.02875 0.144   0.2405  0.805   0.815   0.802   0.82    0.5926  0.6914
  0.455   0.48   ]
 [0.01375 0.1265  0.2245  0.902   0.847   0.859   0.83    0.6588  0.7294
  0.505   

In [20]:
# DEFINE FUNCTIONS TO BE USED TO EVALUATE MODELS AND CONVERT DATA

def get_accuracy(actual, predicted):
    return 1 - sum(predicted != actual) / float(len(actual))

In [21]:
def get_average_error(predicted, actual):
    errors = []
    for p, a in zip(predicted, actual):
        error = abs(p-a)
        errors.append(error)
    return list(df(np.vstack(errors)).mean().round(3))

In [22]:
def get_percent_error(predicted, actual):
    errors = []
    for p, a in zip(predicted, actual):
        error = abs(p-a)/a
        errors.append(error)
    return list(df(np.vstack(errors)).mean().round(3))

In [23]:
# Convert dummy variable vectors to original value.

def value_decoder(data):
    column_list = []
    for n in np.arange(0,len(data)):
        if sum(data[n,:]) == 0:
            column_list.append(0)
        for column in np.arange(0,len(data[0])):
            if data[n,column] != 0:
                column_list.append(column)
    return np.array(column_list)

In [24]:
# Creat unscaler to returns data back to its original scale

# fisher_2h, fisher_24h, fisher_7d, DOI_1day, gloss_1day, DOI_7d, gloss_7d, Amtec_afterScratch, 
# Amtec_afterReflow, Amtect_afterScratchRel6600, Amtec_afterReflowRel6600


# fisher_2h row1 is 23. Scaled to 200 -> 0.115
# fisher_24h row1 is 72.95. Scaled to 200 -> 0.36475
# fisher_7d row1 is 93.9. Scaled to 200 -> 0.4695
# DOI_1d row1 is 93.3. Scaled to 100 -> 0.933
# gloss_1d row1 is 84.2. Scaled to 100 -> 0.842
# DOI_7d row1 is 90.8. Scaled to 100 -> 0.908
# gloss_7d row1 is 85.2. Scaled to 100 -> 0.852
# Amtec_afterScratch row1 is 43.02. Scaled to 100 -> 0.4302
# Amtec_afterReflow row1 is 61.63. Scaled to 100 -> 0.6163
# Amtect_afterScratchRel6600 row1 is 0.73. Scaled to 2 -> 0.365
# Amtec_afterReflowRel6600 row1 is 0.89. Scaled to 2 -> 0.445

def unscaler(data):
    if type(data) == pd.DataFrame:
        data['fisher_2h'] = (data['fisher_2h']*200).round(2)
        data['fisher_24h'] = (data['fisher_24h']*200).round(2)
        data['fisher_7d'] = (data['fisher_7d']*200).round(2)
        data['DOI_1d'] = (data['DOI_1d']*100).round(2)
        data['gloss_1d'] = (data['gloss_1d']*100).round(2)
        data['DOI_7d'] = (data['DOI_7d']*100).round(2)
        data['gloss_7d'] = (data['gloss_7d']*100).round(2)
        data['Amtec_afterScratch'] = (data['Amtec_afterScratch']*100).round(2)
        data['Amtec_afterReflow'] = (data['Amtec_afterReflow']*100).round(2)
        data['Amtect_afterScratchRel6600'] = (data['Amtect_afterScratchRel6600']*2).round(2)
        data['Amtec_afterReflowRel6600'] = (data['Amtec_afterReflowRel6600']*2).round(2)
        
    elif type(data) == np.ndarray:
        data[:,0] = (data[:,0]*200).round(2) # Fisher2h... not sure about 200 here. Or indexer.
        data[:,1] = (data[:,1]*200).round(2) # Fisher24h... not sure about 200 here. Or indexer.
        data[:,2] = (data[:,2]*200).round(2) # Fisher7d... not sure about 200 here. Or indexer.
        data[:,3] = (data[:,3]*100).round(2) # DOI_1d
        data[:,4] = (data[:,4]*100).round(2) # gloss_1d
        data[:,5] = (data[:,5]*100).round(2) # DOI_7d
        data[:,6] = (data[:,6]*100).round(2) # gloss_7d
        data[:,7] = (data[:,7]*100).round(2) # Amtec_afterScratch
        data[:,8] = (data[:,8]*100).round(2) # Amtec_afterReflow
        data[:,9] = (data[:,9]*2).round(2) # Amtect_afterScratchRel6600
        data[:,10] = (data[:,10]*2).round(2) # Amtec_afterReflowhRel6600
        
        data = df(data, columns = ['fisher_2h', 'fisher_24h', 'fisher_7d',
                                   'DOI_1d', 'gloss_1d', 'DOI_7d',
                                  'gloss_7d', 'Amtec_afterScratch',
                                  'Amtec_afterReflow', 'Amtect_afterScratchRel6600',
                                  'Amtec_afterReflowRel6600'])
    return data

In [25]:
# Defin kfold model trainer. Only continous variables now.
# Categorical variables can be added later.

def kfold_model_trainer(x_data, y_data, model, model_type, seed,
                        num_splits, size_of_batches, n_epochs):

    kf = KFold(n_splits = num_splits, shuffle = True, random_state = seed)

    avg_errors = []
    per_errors = []
    accuracies = []
    count = 0

    if model_type == 'continuous':
        for train_index, test_index in kf.split(x_data, y_data):
            count = count + 1

            X_train, X_test = x_data[train_index], x_data[test_index]
            Y_train, Y_test = y_data[train_index], y_data[test_index]
            shuffle([X_train, Y_train])
            
            Y_test = unscaler(Y_test)

            np.random.seed(seed)
            model.fit(X_train, Y_train,
                      batch_size = size_of_batches,
                      epochs = n_epochs)
            pred_values = unscaler(model.predict(X_test))

            avg_errors.append(get_average_error(pred_values.values,
                                                Y_test.values))
            per_errors.append(get_percent_error(pred_values.values,
                                                 Y_test.values))

            average_err = df(avg_errors, columns = ['fisher_2h', 'fisher_24h', 'fisher_7d',
                                                    'DOI_1d', 'gloss_1d', 'DOI_7d',
                                                    'gloss_7d', 'Amtec_afterScratch',
                                                    'Amtec_afterReflow', 'Amtect_afterScratchRel6600',
                                                    'Amtec_afterReflowRel6600'])
            percent_err = df(per_errors, columns = ['fisher_2h', 'fisher_24h', 'fisher_7d',
                                                    'DOI_1d', 'gloss_1d', 'DOI_7d',
                                                    'gloss_7d', 'Amtec_afterScratch',
                                                    'Amtec_afterReflow', 'Amtect_afterScratchRel6600',
                                                    'Amtec_afterReflowRel6600'])
        return [average_err, percent_err]


In [ ]:
# Fit the NN Continuous Model

seed = 0

continuous_model = Sequential()
# add input and first hidden layer
continuous_model.add(Dense(units = 68,
                           input_dim = len(X_checked[0]),
                           kernel_initializer = 'uniform',
                           activation = 'relu'))
'''# add second hidden layer
continuous_model.add(Dense(units = 102,
                           kernel_initializer = 'uniform',
                           activation = 'relu'))'''
# add output layer
continuous_model.add(Dense(units = 11,
                           kernel_initializer = 'uniform',
                           activation = 'relu'))
# compile model
continuous_model.compile(loss = 'mean_squared_error',
                         optimizer = 'Adam',
                         metrics = ['mae'])

# train and save model
con_model_stats = kfold_model_trainer(x_data = X_checked,
                                      y_data = Y_checked,
                                      model = continuous_model,
                                      model_type = 'continuous',
                                      seed = seed,
                                      num_splits = 20,
                                      size_of_batches = 3,
                                      n_epochs = 200)
con_avg_err = con_model_stats[0]
con_perc_err = con_model_stats[1]

continuous_model.fit(X_checked, Y_checked,
                     batch_size = 3, epochs = 200)

continuous_model.save('Axalta_kfold_continuous_network_all_SAPPER.h5')